In [1]:
import mrcnn.utils
import mrcnn.config
import mrcnn.model
import os
import random
import xml.etree.ElementTree as ET
import xml
import numpy as np


from os import listdir
from os.path import isfile, join

In [31]:
class komiDataSet(mrcnn.utils.Dataset):

    def load_dataset(self, is_train = True):
        self.add_class("dataset", 1, 'face')

        images_dir = r'C:\Users\brunolopez\Downloads\komi_analysis_pascal\JPEGImages'
        annotations_dir = r'C:\Users\brunolopez\Downloads\komi_analysis_pascal\Annotations'

        files = [file for file in os.listdir(images_dir)]
        random.shuffle(files) #randomly shuffle the files

        #Get the length of the files and navigate 80 percent for training
        split = 0.7    
        split_index = int(np.floor(len(files) * split))
        
        if is_train:
            files = files[:split_index]
        else:
            files = files[split_index:]

        for filename in files:
            image_id = filename.split('.jpg')[0] #split at the
            #contruct the image path 
            img_path =  images_dir + '/' + filename
            ann_path = annotations_dir + '/' +image_id + '.xml'

            self.add_image('dataset', image_id = image_id, path = img_path, annotation = ann_path)

    def extract_boxes(self, filename):
	    tree = xml.etree.ElementTree.parse(filename)

	    root = tree.getroot()

	    boxes = list()
	    for box in root.findall('.//bndbox'):
		    xmin = int(float(box.find('xmin').text))
		    ymin = int(float(box.find('ymin').text))
		    xmax = int(float(box.find('xmax').text))
		    ymax = int(float(box.find('ymax').text))
		    coors = [xmin, ymin, xmax, ymax]
		    boxes.append(coors)

	    width = int(root.find('.//size/width').text)
	    height = int(root.find('.//size/height').text)
	    return boxes, width, height
    
    def load_mask(self, image_id):
	    info = self.image_info[image_id]
	    path = info['annotation']
	    boxes, w, h = self.extract_boxes(path)
	    masks = np.zeros([h, w, len(boxes)], dtype='uint8')
	    class_ids = list()
	    for i in range(len(boxes)):
		    box = boxes[i]
		    row_s, row_e = box[1], box[3]
		    col_s, col_e = box[0], box[2]
		    masks[row_s:row_e, col_s:col_e, i] = 1
		    class_ids.append(self.class_names.index('face'))
	    return masks, np.asarray(class_ids, dtype='int32')

In [33]:
train_set = komiDataSet()
train_set.load_dataset(is_train=True)
train_set.prepare()

In [34]:
valid_dataset = komiDataSet()
valid_dataset.load_dataset(is_train=False)
valid_dataset.prepare()

In [35]:
class KomiConfig(mrcnn.config.Config):
    NAME = "komi_cfg"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    NUM_CLASSES = 2

    STEPS_PER_EPOCH = 131

In [36]:
model = mrcnn.model.MaskRCNN(mode='training', 
                             model_dir= r'C:\Users\brunolopez\mldata\manga_scrape\komi_weights', 
                             config= KomiConfig())

In [37]:
coco_model = r'C:\Users\brunolopez\mldata\manga_scrape\mask_rcnn_coco.h5'

In [38]:
model.load_weights(filepath=coco_model, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [39]:
model.train(train_dataset=train_set, 
            val_dataset=valid_dataset, 
            learning_rate=KomiConfig().LEARNING_RATE, 
            epochs=10, 
            layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: C:\Users\brunolopez\mldata\manga_scrape\komi_weights\komi_cfg20230228T0137\mask_rcnn_komi_cfg_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDist

ValueError: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type.